In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import *   #lit,col,expr,regexp_extract,regexp_replace,current_date, current_timestamp,date_format
from pyspark.sql.types import StructField,StringType,StructType,DoubleType,LongType,IntegerType,FloatType,TimestampType,DataType
import pyspark

In [2]:
import psycopg2

In [3]:
spark= SparkSession.builder.appName("DataFrame").getOrCreate()

In [4]:
host = "pg.pg4e.com"
port = "5432"
database = "pg4e_e75e85dc24"
user = "pg4e_e75e85dc24"
password = "pg4e_p_e990e39aa8fe644"

In [5]:
jdbc_url = f"jdbc:postgresql://{host}:{port}/{database}"
connection_properties = {
    "user": user,
    "password": password,
    "driver": "org.postgresql.Driver"
}

In [6]:
df = spark.read \
    .jdbc(url=jdbc_url, table="docs03", properties=connection_properties)


In [8]:

df.createOrReplaceTempView("df")


In [59]:
df2=spark.sql("SELECT distinct(id,doc) as array, array('casa',1) as set from df")
df2.createOrReplaceTempView("df2")

In [40]:
from pyspark.sql.types import *
    

In [41]:

#Creo funciones dentro de spark
def capitalize(string):
    if type(string)!=str:
        return(None)
    else:
        return(" ".join(string))


In [43]:
#Registro y creo funcion de prueba
spark.udf.register("capitalize",capitalize,StringType())
#Ejecuto funcion
spark.sql("SELECT capitalize(doc) FROM df").show(5)

+--------------------+
|     capitalize(doc)|
+--------------------+
|W e   c a l l   t...|
|h a v e   v e r y...|
|P y t h o n   p r...|
|y o u   w r i t e...|
|y o u   c a l l e...|
+--------------------+
only showing top 5 rows



In [68]:
#Manejo arrays
spark.sql("SELECT EXPLODE(set) as c1,array FROM df2").show(5)
#Busco en Array
spark.sql("SELECT array_contains(set,'casa') as c1,array FROM df2").show(5)

+----+-----------------+
|  c1|            array|
+----+-----------------+
|casa|{396, Neon 10385}|
|   1|{396, Neon 10385}|
|casa|{542, Neon 10531}|
|   1|{542, Neon 10531}|
|casa|{624, Neon 10613}|
+----+-----------------+
only showing top 5 rows



In [79]:
#Funciones Lambda a los arrays
spark.sql("SELECT transform(set, element -> initcap(element)) as c1,array FROM df2").show(5)
spark.sql("SELECT filter(set, element -> element='casa') as c1,array FROM df2").show(5)

+---------+-----------------+
|       c1|            array|
+---------+-----------------+
|[Casa, 1]|{396, Neon 10385}|
|[Casa, 1]|{542, Neon 10531}|
|[Casa, 1]|{624, Neon 10613}|
|[Casa, 1]|{657, Neon 10646}|
|[Casa, 1]|{683, Neon 10672}|
+---------+-----------------+
only showing top 5 rows

+------+-----------------+
|    c1|            array|
+------+-----------------+
|[casa]|{396, Neon 10385}|
|[casa]|{542, Neon 10531}|
|[casa]|{624, Neon 10613}|
|[casa]|{657, Neon 10646}|
|[casa]|{683, Neon 10672}|
+------+-----------------+
only showing top 5 rows

